## Setup

In [ ]:
import pandas as pd
import json
import numpy as np
from copy import deepcopy

In [ ]:
URL = "https://pocketfrogs.fandom.com/api.php?action=parse&format=json&page=Weekly%20Sets"
tables = pd.read_html(URL)

In [ ]:
for i in range(len(tables)):
    tables[i] = tables[i].drop([tables[i].columns[3], tables[i].columns[4]], axis=1)
    for j in range(len(tables[i].columns)):
        tables[i] = tables[i].rename(columns={tables[i].columns[j]: tables[i].columns[j].strip('\\n')})
        for k in range(len(tables[i].loc[:,tables[i].columns[j]])):
            tables[i].loc[k,tables[i].columns[j]] = tables[i].loc[k,tables[i].columns[j]].strip('\\n')

In [ ]:
def parenReplacement(frontString, inParen, backString, parenNum):
    if (parenNum == 1):
        parens = inParen.split(", ")
        returnString = ""
        if ((frontString == "") or (frontString[0] != " ")):
            frontString = " " + frontString
        for i in range(len(parens)):
            returnString += (frontString + parens[i] + backString + ",")
    else:
        for i in range(parenNum - 1):  
            curString = inParen
            k = curString.find("(")
            cFind = inParen.find(",")
            if ((k < cFind) or (cFind == -1)):
                start = 0
                tempString = curString[:k]
            else:
                while(curString[cFind:].find(",") < k-cFind):
                    cFind += curString[cFind:].find(",") + 1
                start = cFind
                tempString = curString[cFind:k]
            l = curString[k:].find(")")
            r = curString[k+l:].find(",")
            if (r == -1):
                r = len(curString) - k - l
            if ((curString[k+l:].find("(") < curString[k+l:].find(",")) and (curString[k+l:].find("(") != -1)):
                r = 0
                while (curString[k+l+r+1:].find("(") <= curString[k+l+r+1:].find(",")):
                    if (curString[k+l+r+1:].find(",") != -1):
                        r += curString[k+l+r+1:].find(",") + 1
                    else:
                        r = len(curString)-k-l
                        break
            inParen = curString[:start] + parenReplacement(tempString, curString[k+1:k+l], curString[k+l+1:k+l+r], 1) + curString[k+l+r+1:]
            if (inParen[-1] == ","):
                inParen = inParen[:-1]   
            if (inParen[0] == " "):
                inParen = inParen[1:]       
        returnString = parenReplacement(frontString, inParen, backString, 1)
    return returnString

In [ ]:
import os.path
import json
if (not os.path.isfile("completed_sets.json")):
    with open("completed_sets.json", "w") as f:
            json.dump([], f)
if (not os.path.isfile("mailbox.json")):
    with open("mailbox.json", "w") as f:
            json.dump([], f)
if (not os.path.isfile("habitats.json")):
    with open("habitats.json", "w") as f:
            json.dump({}, f)

## Handle Edge Cases

In [ ]:
for i in range(len(tables)):
    for j in range(len(tables[i].loc[:, "Frogs Required"])):    
        if (tables[i].loc[j, "Set Name"] == "Heat Wave"):
            tables[i].loc[j, "Frogs Required"] = tables[i].loc[j, "Frogs Required"][:81]
        if (tables[i].loc[j, "Set Name"] == "Snakes!"):
            while (tables[i].loc[j, "Frogs Required"].find("(") != -1):
                tables[i].loc[j, "Frogs Required"] = tables[i].loc[j, "Frogs Required"][:tables[i].loc[j, "Frogs Required"].find("(")-1] + tables[i].loc[j, "Frogs Required"][tables[i].loc[j, "Frogs Required"].find(")")+1:]
        if (tables[i].loc[j, "Set Name"] == "Teen Ninja Frogs"):
            while (tables[i].loc[j, "Frogs Required"].find("(") != -1):
                tables[i].loc[j, "Frogs Required"] = tables[i].loc[j, "Frogs Required"][:tables[i].loc[j, "Frogs Required"].find("(")-1] + tables[i].loc[j, "Frogs Required"][tables[i].loc[j, "Frogs Required"].find(")")+1:]
            tables[i].loc[j, "Frogs Required"] = "(" + tables[i].loc[j, "Frogs Required"][:-5] + ")" + tables[i].loc[j, "Frogs Required"][-5:]
        if (tables[i].loc[j, "Set Name"] == "Soda Pops"):
            while (tables[i].loc[j, "Frogs Required"].find("(") != -1):
                tables[i].loc[j, "Frogs Required"] = tables[i].loc[j, "Frogs Required"][:tables[i].loc[j, "Frogs Required"].find("(")-1] + tables[i].loc[j, "Frogs Required"][tables[i].loc[j, "Frogs Required"].find(")")+1:]

In [ ]:
for i in range(len(tables)):
    for j in range(len(tables[i].loc[:, "Frogs Required"])):
        while (tables[i].loc[j, "Frogs Required"].find(" and ") != -1):
            ind = tables[i].loc[j, "Frogs Required"].find(" and ")
            if (tables[i].loc[j, "Frogs Required"][ind-1] == ","):
                tables[i].loc[j, "Frogs Required"] = tables[i].loc[j, "Frogs Required"][:ind] + " " + tables[i].loc[j, "Frogs Required"][ind + 5:]
            else:
                tables[i].loc[j, "Frogs Required"] = tables[i].loc[j, "Frogs Required"][:ind] + ", " + tables[i].loc[j, "Frogs Required"][ind + 5:]
        while (tables[i].loc[j, "Frogs Required"].find(" & ") != -1):
            ind = tables[i].loc[j, "Frogs Required"].find(" & ")
            if (tables[i].loc[j, "Frogs Required"][ind-1] == ","):
                tables[i].loc[j, "Frogs Required"] = tables[i].loc[j, "Frogs Required"][:ind] + " " + tables[i].loc[j, "Frogs Required"][ind + 3:]
            else:
                tables[i].loc[j, "Frogs Required"] = tables[i].loc[j, "Frogs Required"][:ind] + ", " + tables[i].loc[j, "Frogs Required"][ind + 3:]
        while (tables[i].loc[j, "Frogs Required"].find(";") != -1):
            ind = tables[i].loc[j, "Frogs Required"].find(";")
            tables[i].loc[j, "Frogs Required"] = tables[i].loc[j, "Frogs Required"][:ind] + "," + tables[i].loc[j, "Frogs Required"][ind+1:]

## Handle Parenthesis

In [ ]:
i = 0
j = 0
k = 0
while(i < len(tables)):
    while(j < len(tables[i].loc[:,"Frogs Required"])):
        # print(i, j)
        # print(tables[i].loc[j, "Frogs Required"])
        frogString = ""
        tempString = ""
        while(k < len(tables[i].loc[j, "Frogs Required"])):
            # if (j == 37):
            #     print(k)
            #     print(tables[i].loc[j, "Frogs Required"])
            #     print(tables[i].loc[j, "Frogs Required"][k])
            curString = tables[i].loc[j, "Frogs Required"]
            curChar = curString[k]
            if (curChar == "("):
                l = curString[k:].find(")")
                if(len(curString[k+1:k+l].split("(")) == 1):
                    r = curString[k+l:].find(",")
                    if (r == -1):
                        r = len(curString) - k - l
                    if ((curString[k+l:].find("(") < curString[k+l:].find(",")) and (curString[k+l:].find("(") != -1)):
                        r = curString[k+l:].find("(")
                        r += curString[k+l+r:].find(")")
                        if (curString[k+l+r:].find(",") == -1):
                            r = len(curString)-k-l
                        else:
                            r += curString[k+l+r:].find(",") + 1
                        # if (j == 37):
                        #     print(curString)
                        #     print(k)
                        #     print(l)
                        #     print(r)
                        #     print(len(curString))
                        #     print(curString[k+l+r+1:].find(","))
                        # while (curString[k+l+r+1:].find(")") <= curString[k+l+r+1:].find(",")):
                        #     if (curString[k+l+r+1:].find(",") != -1):
                        #         r += curString[k+l+r+1:].find(",") + 1
                        #     else:
                        #         r = len(curString)-k-l
                        #         break
                    tables[i].loc[j, "Frogs Required"] = curString[:k-len(tempString)] + parenReplacement(tempString, curString[k+1:k+l], curString[k+l+1:k+l+r], 1) + curString[k+l+r+1:]
                    if (tables[i].loc[j, "Frogs Required"][-1] == ","):
                        tables[i].loc[j, "Frogs Required"] = tables[i].loc[j, "Frogs Required"][:-1]
                    k -= len(tempString)
                    tempString = ""
                    continue
                else:
                    while (curString[k+1:k+l].count("(") != curString[k+1:k+l].count(")")):
                        l += curString[k+l+1:].find(")")+1
                    r = curString[k+l:].find(",")
                    if (r == -1):
                        r = len(curString) - k - l
                    if ((curString[k+l:].find("(") < curString[k+l:].find(",")) and (curString[k+l:].find("(") != -1)):
                        r = 0
                        while (curString[k+l+r+1:].find("(") <= curString[k+l+r+1:].find(",")):
                            if (curString[k+l+r+1:].find(",") != -1):
                                r += curString[k+l+r+1:].find(",") + 1
                            else:
                                r = len(curString)-k-l
                                break
                    tables[i].loc[j, "Frogs Required"] = curString[:k-len(tempString)] + parenReplacement(tempString, curString[k+1:k+l], curString[k+l+1:k+l+r], curString[k+1:k+l].count("(")+1) + curString[k+l+r+1:]
                    if (tables[i].loc[j, "Frogs Required"][-1] == ","):
                        tables[i].loc[j, "Frogs Required"] = tables[i].loc[j, "Frogs Required"][:-1]
                    k -= len(tempString)
                    tempString = ""
                    continue
            tempString += curChar
            k += 1
            if (curChar == ","):
                frogString += tempString
                tempString = ""
        frogString += tempString
        tempString = ""
        if frogString[0] == " ":
            frogString = frogString[1:]
        if frogString[-1] == ",":
            frogString = frogString[:-1]
        while(frogString.find(",,") != -1):
            ind = frogString.find(",,")
            frogString = frogString[:ind] + frogString[ind+1:]
        # print(frogString)
        tables[i].loc[j, "Frogs Required"] = frogString
        frogString = ""
        j += 1
        k = 0
    i += 1
    j = 0
    k = 0

## Handle Multipliers

In [ ]:
for i in range(len(tables)):
    for j in range(len(tables[i].loc[:, "Frogs Required"])):
        curList = (" " + tables[i].loc[j, "Frogs Required"]).split(",")
        for k in range(len(curList)):
            curItem = curList[k]
            try:
                nums = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
                if (curItem.find("X") != -1):
                    temp = curItem.split("X")
                else:
                    temp = curItem.split("x")
                if (len(temp) == 1):
                    raise ValueError
                else:
                    multed = False
                    for p in range(len(temp) - 1):
                        if (temp[p][-1] in nums):
                            mult = temp[p][-1]
                            multed = True
                            break
                    if (not multed):
                        raise ValueError
                    multInd = curItem.find(mult)
                    mult = int(mult)
                    frog = curItem[:multInd - 1] + curItem[multInd + 2:]
                # mult = int(curItem[1])
                # frog = curItem[3:]
                # # print(mult)
                # # print(frog)
                    curItem = ""
                    for l in range(mult):
                        curItem += (frog + ",")
                    curList[k] = curItem
            except ValueError:
                curList[k] = curItem + ","
        frogString = ""
        for r in range(len(curList)):
            frogString += curList[r]
        if frogString[0] == " ":
            frogString = frogString[1:]
        if frogString[-1] == ",":
            frogString = frogString[:-1]
        tables[i].loc[j, "Frogs Required"] = frogString

## Split Into Dictionary

In [ ]:
frogDict = {}
fullFrogDict = {}
with open("completed_sets.json", 'r') as f:
    completedSets = json.load(f)
if (type(completedSets) != list):
    completedSets = json.loads(completedSets)

for i in range(len(tables)):
    for j in range(len(tables[i].loc[:,"Frogs Required"])):
        # print(i, j)
        curList = (" " + tables[i].loc[j, "Frogs Required"]).split(",")
        if (tables[i].loc[j, "Set Name"] in completedSets):
            continue
        for k in range(len(curList)):
            curItem = curList[k].split(" ")
            if (curItem[0] == ""):
                curItem = curItem[1:]
            if (curItem[2] not in frogDict.keys()):
                frogDict[curItem[2]] = {}
                frogDict[curItem[2]][curItem[0]] = 1
                frogDict[curItem[2]][curItem[1]] = 1
            else:
                if (curItem[0] not in frogDict[curItem[2]].keys()):
                    frogDict[curItem[2]][curItem[0]] = 1
                else:
                    frogDict[curItem[2]][curItem[0]] += 1
                if (curItem[1] not in frogDict[curItem[2]].keys()):
                    frogDict[curItem[2]][curItem[1]] = 1
                else:
                    frogDict[curItem[2]][curItem[1]] += 1
            
            fullFrogName = curItem[0] + " " + curItem[1]
            if (curItem[2] not in fullFrogDict.keys()):
                fullFrogDict[curItem[2]] = {}
                fullFrogDict[curItem[2]][fullFrogName] = 1
            elif (fullFrogName not in fullFrogDict[curItem[2]].keys()):
                fullFrogDict[curItem[2]][fullFrogName] = 1
            else:
                fullFrogDict[curItem[2]][fullFrogName] += 1


In [ ]:
addDict = fullFrogDict["Forame"]
del fullFrogDict["Forame"]
for key in addDict.keys():
    if key in list(fullFrogDict["Foramen"].keys()):
        fullFrogDict["Foramen"][key] += addDict[key]
    else:
        fullFrogDict["Foramen"][key] = addDict[key]

In [ ]:
def Set_Completed(setName, checkFrogs=False):
    with open("completed_sets.json", 'r') as f:
        completedSets = json.load(f)
    if (type(completedSets) != list):
        completedSets = json.loads(completedSets)

    if(setName in completedSets):
        print("Set Already Completed")
        return 

    curList = None
    completedSets.append(setName)
    for i in range(len(tables)):
        for j in range(len(tables[i].loc[:,"Set Name"])):
            if tables[i].loc[j, "Set Name"] == setName:
                print("Set Found")
                curList = (" " + tables[i].loc[j, "Frogs Required"]).split(",")
                for k in range(len(curList)):
                    curList[k] = curList[k].strip()
                    curItem = curList[k].split(" ")
                    frogDict[curItem[2]][curItem[0]] -= 1
                    frogDict[curItem[2]][curItem[1]] -= 1
                    fullFrogDict[curItem[2]][curItem[0] + " " + curItem[1]] -= 1
                    if (fullFrogDict[curItem[2]][curItem[0] + " " + curItem[1]] == 0):
                        del fullFrogDict[curItem[2]][curItem[0] + " " + curItem[1]]
                        if (len(fullFrogDict[curItem[2]].keys()) == 0):
                            print("00")
                            del fullFrogDict[curItem[2]]

    with open("completed_sets.json", "w") as f:
        json.dump(completedSets, f)

    del completedSets

    if (checkFrogs):
        if (curList is not None):
            fc = fullCoverage(addFrog=curList)
            curSell = []
            curKeep = []
            sellOther = []
            for frog in fc:
                if ((frog[1] != "N") and (frog[1] != ["N"])):
                    sellOther.append(frog)
                else:
                    curSell.append(frog[0])
            for frog in curList:
                if frog not in curSell:
                    curKeep.append(frog)

            # transfer = []
            # for frog in curKeep:
            #     for frogHab in sellOther:
            #         if ((frog == frogHab[0]) and ((frogHab[1] == "N") or (frogHab[1][0] == "N"))):
            #             transfer.append(frog)
            #             sellOther.remove(frogHab)
            #             break
            # for frog in transfer:
            #     curKeep.remove(frog)
            #     curSell.append(frog)

            print("Keep: " + str(curKeep))
            print("Sell (This Set): " + str(curSell))
            print("Sell (Other): " + str(sellOther))
            return 

## Find Frog

In [ ]:
def keepOrSell(frog):
    frogList = frog.split(" ")
    if (frogList[2] not in frogDict.keys()):
        return "Sell"
    else:
        # fD = frogDict[frogList[2]]
        # if ((frogList[0] in fD.keys()) and (fD[frogList[0]] != 0)):
        #     return "Keep"
        # if ((frogList[1] in fD.keys()) and (fD[frogList[1]] != 0)):
        #     return "Keep"
        # return "Sell"

        newList = fullCoverage(addFrog=frog)
        if (len(newList) == 0):
            return "Keep"
        elif ((len(newList) == 1) and (newList[0][0] == frog)):
            return "Sell"
        else:
            return newList
        

## Mailbox and Habitat Code

In [ ]:
def addToMailbox(frog):
    with open("mailbox.json", 'r') as f:
        mailbox = json.load(f)
    if (type(mailbox) != list):
        mailbox = json.loads(mailbox)

    mailbox.append(frog)
    # print("Keep or Sell Output: " + keepOrSell(frog))

    with open("mailbox.json", "w") as f:
        json.dump(mailbox, f)

In [ ]:
def removeFromMailbox(frog):
    with open("mailbox.json", 'r') as f:
        mailbox = json.load(f)
    if (type(mailbox) != list):
        mailbox = json.loads(mailbox)

    try:
        mailbox.remove(frog)

        with open("mailbox.json", "w") as f:
            json.dump(mailbox, f)
        print(frog + " removed from Mailbox")
    except (ValueError):
        print(frog + " not in Mailbox")
        with open("habitats.json", 'r') as f:
            habitats = json.load(f)
        if (type(habitats) != dict):
            habitats = json.loads(habitats)

        for hab in list(habitats.keys()):
            if (frog in habitats[hab]):
                print(frog + " found in habitat " + hab)

In [ ]:
def addToHabitat(frog, habNum, check=True):
    with open("habitats.json", 'r') as f:
        habitats = json.load(f)
    if (type(habitats) != dict):
        habitats = json.loads(habitats)
    with open("mailbox.json", 'r') as f:
        mailbox = json.load(f)
    if (type(mailbox) != list):
        mailbox = json.loads(mailbox)

    if (check):
        found = False
        hk = list(habitats.keys())
        for i in range(len(hk)):
            for frog2 in habitats[hk[i]]:
                if (habitats[hk[i]].count(frog) > 1):
                    found=hk[i]
                for j in range(len(hk)):
                    if (i != j):
                        if (frog == frog2):
                            found=hk[i]
            if (mailbox.count(frog) > 0):
                found="M"
        
        if (found != False):
            if (found == "M"):
                x = input("Frog found in Mailbox. Would you like to move frog instead? (Y/N)")
            else:
                x = input("Frog found in Habitat " + found + ". Would you like to move frog instead? (Y/N)")
            x = x.lower()
            if ((x == "y") or (x == "yes") or (x == "ye") or (x == "yeah") or (x == "sure")):
                if (found == "M"):
                    removeFromMailbox(frog)
                    with open("mailbox.json", 'r') as f:
                        mailbox = json.load(f)
                    if (type(mailbox) != list):
                        mailbox = json.loads(mailbox)
                else:
                    removeFromHabitat(frog, found)
                    with open("habitats.json", 'r') as f:
                        habitats = json.load(f)
                    if (type(habitats) != dict):
                        habitats = json.loads(habitats)
            if ((x == "c") or (x == "cancel") or (x == None) or (x == "")):
                return

    # print("Keep or Sell Output: " + keepOrSell(frog))
    habitats[habNum].append(frog)

    with open("habitats.json", "w") as f:
        json.dump(habitats, f, indent = 0)

In [ ]:
def removeFromHabitat(frog, habNum):
    with open("habitats.json", 'r') as f:
        habitats = json.load(f)
    if (type(habitats) != dict):
        habitats = json.loads(habitats)

    try:
        habitats[habNum].remove(frog)

        with open("habitats.json", "w") as f:
            json.dump(habitats, f)
        print(frog + " removed from Habitat " + habNum)
    except (ValueError):
        print(frog + " not in Habitat " + habNum)
        for hab in list(habitats.keys()):
            if (frog in habitats[hab]):
                print(frog + " found in Habitat " + hab)

        with open("mailbox.json", 'r') as f:
            mailbox = json.load(f)
        if (type(mailbox) != list):
            mailbox = json.loads(mailbox)

        if frog in mailbox:
            print(frog + " found in Mailbox")

In [ ]:
def moveHabitats(frog, habFrom, habTo):
    addToHabitat(frog, habTo, check=False)
    removeFromHabitat(frog, habFrom)

In [ ]:
def addHabitat(habNum):
    with open("habitats.json", 'r') as f:
        habitats = json.load(f)
    if (type(habitats) != dict):
        habitats = json.loads(habitats)

    habitats[habNum] = []

    with open("habitats.json", "w") as f:
        json.dump(habitats, f, indent = 0)

In [ ]:
def moveMailboxToHabitat(frog, habNum):
    removeFromMailbox(frog)
    addToHabitat(frog, habNum)

In [ ]:
def sellOrAddToMailbox(frog):
    kOS = keepOrSell(frog)
    if (kOS == "Keep"):
        addToMailbox(frog)
        return "Keep"
    elif (kOS == "Sell"):
        return "Sell"
    else:
        if (frog not in list(list(zip(*kOS))[0])):
            addToMailbox(frog)
            print("Added")
        else:
            print("Not Added")
        return kOS

In [ ]:
def moveAll(sourceHab, destHab):
    with open("habitats.json", 'r') as f:
        habitats = json.load(f)
    if (type(habitats) != dict):
        habitats = json.loads(habitats)

    for frog in habitats[sourceHab]:
        moveHabitats(frog, sourceHab, destHab)

In [ ]:
def lastMailboxFrog():
    with open("mailbox.json", 'r') as f:
        mailbox = json.load(f)
    if (type(mailbox) != list):
        mailbox = json.loads(mailbox)

    return mailbox[len(mailbox) - 1]

## Code to Use During Set Completion

In [ ]:
def keepAfterSet(frog, setName):
    with open("completed_sets.json", 'r') as f:
        completedSets = json.load(f)
    if (type(completedSets) != list):
        completedSets = json.loads(completedSets)
    with open("mailbox.json", 'r') as f:
        mailbox = json.load(f)
    if (type(mailbox) != list):
        mailbox = json.loads(mailbox)
    with open("habitats.json", 'r') as f:
        habitats = json.load(f)
    if (type(habitats) != dict):
        habitats = json.loads(habitats)

    if(setName in completedSets):
        print("Set Already Completed")
        return

    completedSets.append(setName)
    global frogDict
    global fullFrogDict
    frogDict2 = deepcopy(frogDict)
    fullFrogDict2 = deepcopy(fullFrogDict)
    found = False
    for i in range(len(tables)):
        for j in range(len(tables[i].loc[:,"Set Name"])):
            if tables[i].loc[j, "Set Name"] == setName:
                print("Set Found")
                found = True
                curList = (" " + tables[i].loc[j, "Frogs Required"]).split(",")
                for k in range(len(curList)):
                    curList[k] = curList[k].strip()
                    curItem = curList[k].split(" ")
                    frogDict[curItem[2]][curItem[0]] -= 1
                    frogDict[curItem[2]][curItem[1]] -= 1
                    fullFrogDict[curItem[2]][curItem[0] + " " + curItem[1]] -= 1

    del completedSets

    if (found):
        frogList = frog.split(" ")
        if (frogList[2] not in frogDict.keys()):
            frogDict = frogDict2
            fullFrogDict = fullFrogDict2
            return "Sell"
        else:
            removed = True
            if (frog in mailbox):
                removed = False
            for hab in habitats:
                if (frog in habitats[hab]):
                    removed = False
            if (removed):
                newList = fullCoverage(addFrog=curList+[frog])
            else:
                newList = fullCoverage(addFrog=curList)
            
            frogDict = frogDict2
            fullFrogDict = fullFrogDict2
            if (len(newList) == 0):
                return "Keep"
            elif (((len(newList) == 1) and (newList[0][0] == frog)) or (frog in list(zip(*newList))[0])):
                return "Sell"
            else:
                return "Keep"
    else:
        return "No Set Found"

In [ ]:
def bestFrog(frog, amnt=False):
    frogSplit = frog.split(" ")

    with open("mailbox.json", 'r') as f:
        mailbox = json.load(f)
    if (type(mailbox) != list):
        mailbox = json.loads(mailbox)
    with open("habitats.json", 'r') as f:
        habitats = json.load(f)
    if (type(habitats) != dict):
        habitats = json.loads(habitats)
    habs = list(habitats.keys())
    habs.reverse()

    bestMatch = ("", 0)
    matchAmount = 0
    for hab in habs:
        for mH in habitats[hab]:
            mHSplit = mH.split(" ")
            if (mHSplit[2] == frogSplit[2]):
                if ((frogSplit[0] == mHSplit[0]) and (frogSplit[1] == mHSplit[1])):
                    if (amnt):
                        return 3
                    return (mH, hab)
                elif (matchAmount == 2):
                    continue
                elif ((frogSplit[0] == mHSplit[0]) or (frogSplit[1] == mHSplit[1])):
                    matchAmount = 2
                    bestMatch = (mH, hab)
                elif (matchAmount == 1):
                    continue
                else:
                    matchAmount = 1
                    bestMatch = (mH, hab)

    for mH in mailbox:
        mHSplit = mH.split(" ")
        if (mHSplit[2] == frogSplit[2]):
            if ((frogSplit[0] == mHSplit[0]) and (frogSplit[1] == mHSplit[1])):
                bestMatch = (mH, "M", mailbox.index(mH))
            elif (matchAmount == 2):
                continue
            elif ((frogSplit[0] == mHSplit[0]) or (frogSplit[1] == mHSplit[1])):
                matchAmount = 2
                bestMatch = (mH, "M", mailbox.index(mH))
            elif (matchAmount == 1):
                continue
            else:
                matchAmount = 1
                bestMatch = (mH, "M", mailbox.index(mH))

    if (amnt):
        return matchAmount
    return bestMatch

In [ ]:
def connectFrogs(frog):
    frogSplit = frog.split(" ")
    relFrogs = fullFrogDict[frogSplit[2]]

    retFrogs = []
    for rF in relFrogs:
        rFS = rF.split(" ")
        if ((rFS[0] == frogSplit[0]) or (rFS[1] == frogSplit[1])):
            retFrogs.append(rF + " " + frogSplit[2])
    return retFrogs

In [ ]:
def findDuplicates():
    with open("mailbox.json", 'r') as f:
        mailbox = json.load(f)
    if (type(mailbox) != list):
        mailbox = json.loads(mailbox)
    with open("habitats.json", 'r') as f:
        habitats = json.load(f)
    if (type(habitats) != dict):
        habitats = json.loads(habitats)

    retList = []
    hk = list(habitats.keys())
    for i in range(len(hk)):
        for frog in habitats[hk[i]]:
            if (habitats[hk[i]].count(frog) > 1):
                retList.append((frog, (hk[i])))
            for j in range(i+1, len(hk)):
                if (habitats[hk[j]].count(frog) > 0):
                    retList.append((frog, (hk[i], hk[j])))
            if (mailbox.count(frog) > 0):
                retList.append((frog, (hk[i], "M")))
    
    for frog in mailbox:
        if (mailbox.count(frog) > 1):
            retList.append(frog, ("M"))
    return retList

In [ ]:
def findInMailbox(frog):
    with open("mailbox.json", 'r') as f:
        mailbox = json.load(f)
    if (type(mailbox) != list):
        mailbox = json.loads(mailbox)

    return (mailbox.index(frog), len(mailbox))

## Graph Coverage Code

In [ ]:
def fullCoverage(addFrog = None, returnNum = None, returnKeep = False, mb = None, hs = None, ffd = None):
    minLockedHab = 16
    lockedFrogs = {}
    unlockedFrogs = {}
    frogLocs = {}

    keepFrogs = []
    sellFrogs = []
    global fullFrogDict

    # if (returnNum == None):
    #     if (addFrog == None):
    #         returnNum = False
    #     else:
    #         returnNum = True

    if (mb == None):
        with open("mailbox.json", 'r') as f:
            mailbox = json.load(f)
        if (type(mailbox) != list):
            mailbox = json.loads(mailbox)
    else:
        mailbox = mb

    if (hs == None):
        with open("habitats.json", 'r') as f:
            habitats = json.load(f)
        if (type(habitats) != dict):
            habitats = json.loads(habitats)
    else:
        habitats = hs

    ###################
    # Prep Frog Lists #
    ###################

    for frog in mailbox:
        frogSplit = frog.split(" ")
        if (frogSplit[2] in list(unlockedFrogs.keys())):
            unlockedFrogs[frogSplit[2]].append(frogSplit[0] + " " + frogSplit[1])
        else:
            unlockedFrogs[frogSplit[2]] = [frogSplit[0] + " " + frogSplit[1]]

        if (frog in list(frogLocs.keys())):
            sellFrogs.append((frog, frogLocs[frog]))
            frogLocs[frog] = ["M"]
        else:
            frogLocs[frog] = ["M"]
    
    for hab in habitats:
        if ((hab == "N") or (int(hab) < minLockedHab)):
            for frog in habitats[hab]:
                frogSplit = frog.split(" ")
                if (frogSplit[2] in list(unlockedFrogs.keys())):
                    unlockedFrogs[frogSplit[2]].append(frogSplit[0] + " " + frogSplit[1])
                else:
                    unlockedFrogs[frogSplit[2]] = [frogSplit[0] + " " + frogSplit[1]]

                if (frog in list(frogLocs.keys())):
                    sellFrogs.append((frog, frogLocs[frog]))
                    frogLocs[frog] = [hab]
                else:
                    frogLocs[frog] = [hab]
        else:
            for frog in habitats[hab]:
                frogSplit = frog.split(" ")
                if (frogSplit[2] in list(lockedFrogs.keys())):
                    lockedFrogs[frogSplit[2]].append(frogSplit[0] + " " + frogSplit[1])
                else:
                    lockedFrogs[frogSplit[2]] = [frogSplit[0] + " " + frogSplit[1]]

                if (frog in list(frogLocs.keys())):
                    sellFrogs.append((frog, frogLocs[frog]))
                    frogLocs[frog] = [hab]
                else:
                    frogLocs[frog] = [hab]

    if (addFrog != None):
        if (type(addFrog) == str):
            addFrogSplit = addFrog.split(" ")
            if (addFrog in list(frogLocs.keys())):
                sellFrogs.append((addFrog, frogLocs[addFrog]))
                frogLocs[addFrog] = ["N"]
            else:
                frogLocs[addFrog] = ["N"]

            if (addFrogSplit[2] in list(unlockedFrogs.keys())):
                unlockedFrogs[addFrogSplit[2]].append(addFrogSplit[0] + " " + addFrogSplit[1])
            else:
                unlockedFrogs[addFrogSplit[2]] = [addFrogSplit[0] + " " + addFrogSplit[1]]
        if (type(addFrog) == list):
            for frog in addFrog:
                frogSplit = frog.split(" ")
                if (frog in list(frogLocs.keys())):
                    sellFrogs.append((frog, frogLocs[frog]))
                    frogLocs[frog] = ["N"]
                else:
                    frogLocs[frog] = ["N"]

                if (frogSplit[2] in list(unlockedFrogs.keys())):
                    unlockedFrogs[frogSplit[2]].append(frogSplit[0] + " " + frogSplit[1])
                else:
                    unlockedFrogs[frogSplit[2]] = [frogSplit[0] + " " + frogSplit[1]]

    ##########################
    # Cycle Through Patterns #
    ##########################
    if (ffd != None):
        tempFfd = deepcopy(fullFrogDict)
        fullFrogDict = ffd
        
    for pattern in list(fullFrogDict.keys()):
        if (pattern not in list(lockedFrogs.keys())):
            continue

        if (pattern not in list(fullFrogDict.keys())):
            if (pattern not in list(unlockedFrogs.keys())):
                mhFrogs = list({frog:[] for frog in lockedFrogs[pattern]}.keys())
            else:
                mhFrogs = list({frog:[] for frog in (lockedFrogs[pattern] + unlockedFrogs[pattern])}.keys())

            for frog in mhFrogs:
                retHab = frogLocs[frog + " " + pattern][0]
                frogLocs[frog + " " + pattern] = frogLocs[frog + " " + pattern][1:]
                if ((retHab == "M") or (retHab == "N") or (int(retHab) < int(minLockedHab))):
                    sellFrogs.append((frog + " " + pattern, retHab))
                else:
                    keepFrogs.append((frog + " " + pattern, retHab))
        else:
            setFrogs = {frog: [] for frog in fullFrogDict[pattern]}
            if (pattern not in list(unlockedFrogs.keys())):
                mhFrogs = {frog:[] for frog in lockedFrogs[pattern]}
            else:
                mhFrogs = {frog:[] for frog in (lockedFrogs[pattern] + unlockedFrogs[pattern])}
            for sF in list(setFrogs.keys()):
                if (fullFrogDict[pattern][sF] == 0):
                    fullFrogDict[pattern].pop(sF, None)
                    continue
                sFSplit = sF.split(" ")
                for mhF in list(mhFrogs.keys()):
                    mhFSplit = mhF.split(" ")
                    if ((sFSplit[0] == mhFSplit[0]) or (sFSplit[1] == mhFSplit[1])):
                        setFrogs[sF].append(mhF)
                        mhFrogs[mhF].append(sF)

            for key in list(setFrogs.keys()):
                if (len(setFrogs[key]) == 0):
                    setFrogs.pop(key)
            for key in list(mhFrogs.keys()):
                if (len(mhFrogs[key]) == 0):
                    mhFrogs.pop(key)
                    frog = key
                    retHab = frogLocs[frog + " " + pattern][0]
                    frogLocs[frog + " " + pattern] = frogLocs[frog + " " + pattern][1:]
                    if ((retHab == "M") or (retHab == "N") or (int(retHab) < int(minLockedHab))):
                        sellFrogs.append((frog + " " + pattern, retHab))
                    else:
                        keepFrogs.append((frog + " " + pattern, retHab))
            if ((len(setFrogs.keys()) == 0) or (len(mhFrogs.keys()) == 0)):
                continue

            if pattern in list(unlockedFrogs.keys()):
                for lF in unlockedFrogs[pattern]:
                    if (lF in mhFrogs.keys()):
                        if lF in mhFrogs[lF]:
                            retHab = frogLocs[lF + " " + pattern][0]
                            frogLocs[lF + " " + pattern] = frogLocs[lF + " " + pattern][1:]
                            keepFrogs.append((lF + " " + pattern, retHab))
                            for sF in mhFrogs[lF]:
                                for mhF in setFrogs[sF]:
                                    if (mhF != lF):
                                        mhFrogs[mhF].remove(sF)
                                setFrogs.pop(sF)
                            mhFrogs.pop(lF)

            for lF in lockedFrogs[pattern]:
                if (lF in mhFrogs.keys()):
                    for sF in mhFrogs[lF]:
                        for mhF in setFrogs[sF]:
                            if (mhF != lF):
                                mhFrogs[mhF].remove(sF)
                        setFrogs.pop(sF)
                    retHab = frogLocs[lF + " " + pattern][0]
                    frogLocs[lF + " " + pattern] = frogLocs[lF + " " + pattern][1:]
                    keepFrogs.append((lF + " " + pattern, retHab))
                    mhFrogs.pop(lF)

            for key in list(setFrogs.keys()):
                if (len(setFrogs[key]) == 0):
                    setFrogs.pop(key)
            for key in list(mhFrogs.keys()):
                if (len(mhFrogs[key]) == 0):
                    mhFrogs.pop(key)
                    frog = key
                    retHab = frogLocs[frog + " " + pattern][0]
                    frogLocs[frog + " " + pattern] = frogLocs[frog + " " + pattern][1:]
                    if ((retHab == "M") or (retHab == "N") or (int(retHab) < int(minLockedHab))):
                        sellFrogs.append((frog + " " + pattern, retHab))
                    else:
                        keepFrogs.append((frog + " " + pattern, retHab))
            if ((len(setFrogs.keys()) == 0) or (len(mhFrogs.keys()) == 0)):
                continue

            ones = True
            while((len(list(setFrogs.keys())) > 0) and (len(list(mhFrogs.keys())) > 0)):
                if (ones):
                    ones = False
                    onesFrogs = []
                    for setF in list(setFrogs.keys()):
                        if ((len(setFrogs[setF]) == 1) and (setFrogs[setF][0] not in onesFrogs)):
                            onesFrogs.append(setFrogs[setF][0])
                            ones = True
                    
                    if (len(onesFrogs) > 0):
                        for lF in onesFrogs:
                            for sF in mhFrogs[lF]:
                                for mhF in setFrogs[sF]:
                                    if (mhF != lF):
                                        mhFrogs[mhF].remove(sF)
                                setFrogs.pop(sF)
                            retHab = frogLocs[lF + " " + pattern][0]
                            frogLocs[lF + " " + pattern] = frogLocs[lF + " " + pattern][1:]
                            keepFrogs.append((lF + " " + pattern, retHab))
                            mhFrogs.pop(lF)

                        for key in list(setFrogs.keys()):
                            if (len(setFrogs[key]) == 0):
                                setFrogs.pop(key)
                        for key in list(mhFrogs.keys()):
                            if (len(mhFrogs[key]) == 0):
                                mhFrogs.pop(key)
                                frog = key
                                retHab = frogLocs[frog + " " + pattern][0]
                                frogLocs[frog + " " + pattern] = frogLocs[frog + " " + pattern][1:]
                                if ((retHab == "M") or (retHab == "N") or (int(retHab) < int(minLockedHab))):
                                    sellFrogs.append((frog + " " + pattern, retHab))
                                else:
                                    keepFrogs.append((frog + " " + pattern, retHab))

                else:
                    ones = True
                    most = 0
                    mostHab = ""
                    mostFrog = ""

                    for mhF in list(mhFrogs.keys()):
                        if (len(mhFrogs[mhF]) > most):
                            most = len(mhFrogs[mhF])
                            mostFrog = mhF
                            mostHab = frogLocs[mhF + " " + pattern]
                        elif ((len(mhFrogs[mhF]) == most) and (mostHab != "M") and (frogLocs[mhF + " " + pattern] == "M")):
                            mostFrog = mhF
                            mostHab = frogLocs[mhF + " " + pattern]     

                    lF = mostFrog
                    for sF in mhFrogs[lF]:
                        for mhF in setFrogs[sF]:
                            if (mhF != lF):
                                mhFrogs[mhF].remove(sF)
                        setFrogs.pop(sF)
                    retHab = frogLocs[lF + " " + pattern][0]
                    frogLocs[lF + " " + pattern] = frogLocs[lF + " " + pattern][1:]
                    keepFrogs.append((lF + " " + pattern, retHab))
                    mhFrogs.pop(lF)

                    for key in list(setFrogs.keys()):
                        if (len(setFrogs[key]) == 0):
                            setFrogs.pop(key)
                    for key in list(mhFrogs.keys()):
                        if (len(mhFrogs[key]) == 0):
                            mhFrogs.pop(key)
                            frog = key
                            retHab = frogLocs[frog + " " + pattern][0]
                            frogLocs[frog + " " + pattern] = frogLocs[frog + " " + pattern][1:]
                            if ((retHab == "M") or (retHab == "N") or (int(retHab) < int(minLockedHab))):
                                sellFrogs.append((frog + " " + pattern, retHab))
                            else:
                                keepFrogs.append((frog + " " + pattern, retHab))  
    
    if (ffd != None):
        fullFrogDict = tempFfd

    if(returnNum):
        return len(keepFrogs)
    elif(returnKeep):
        return keepFrogs
    else:
        return sellFrogs  

In [ ]:
fc = fullCoverage()
fc

In [ ]:
with open("mailbox.json", 'r') as f:
    mailbox = json.load(f)
if (type(mailbox) != list):
    mailbox = json.loads(mailbox)

newFC = []
for frog in fc:
    if (frog[0] in mailbox):
        newFC.append((frog[0], mailbox.index(frog[0])))
print(len(newFC))

for frog in (sorted(newFC, key=lambda tup: tup[1])[-8:]):
    print(frog)
    # removeFromMailbox(frog[0])

## Choose Next Set

In [ ]:
fullCoverage(returnNum = True)

In [ ]:
def nextSet(space=True):
    with open("completed_sets.json", 'r') as f:
        completedSets = json.load(f)
    if (type(completedSets) != list):
        completedSets = json.loads(completedSets)
    with open("habitats.json", 'r') as f:
        habitats = json.load(f)
    if (type(habitats) != dict):
        habitats = json.loads(habitats)
    global fullFrogDict

    bestLen = None
    bestM = None
    bestSet = ""
    bestSetInd = (-1, -1)

    if (not space):
        for i in range(len(tables)):
            for j in range(len(tables[i].loc[:, "Set Name"])):
                if tables[i].loc[j, "Set Name"] in completedSets:
                    continue

                newFrogs = tables[i].loc[j, "Frogs Required"].split(",")
                tempLen = 0
                poss = True
                for nF in newFrogs:
                    nF = nF.strip()
                    if (nF.split(" ")[2] not in list(fullFrogDict.keys())):
                        poss = False
                    tempLen += bestFrog(nF, amnt=True)
                tempLen /= len(newFrogs)
                if (((bestLen is None) or (tempLen > bestLen)) and (poss)):
                    bestLen = tempLen
                    bestSet = tables[i].loc[j, "Set Name"]
                    bestSetInd = (i, j)
        print(bestLen)
        return (bestSet, bestSetInd[0], "Week " + str(len(tables[bestSetInd[0]]) - bestSetInd[1]))

    for i in range(len(tables)):
        for j in range(len(tables[i].loc[:, "Set Name"])):
            if tables[i].loc[j, "Set Name"] in completedSets:
                continue

            newFrogs = tables[i].loc[j, "Frogs Required"].split(",")

            allFrogs = True
            for nF in newFrogs:
                nF = nF.strip()
                thisFrog = False
                for hF in sum(habitats.values(), []):
                    thisFrog = thisFrog or (nF.split(" ")[2] == hF.split(" ")[2])
                allFrogs = allFrogs and thisFrog
            if (not allFrogs):
                continue

            tempHab = deepcopy(habitats)
            tempFrogDict = deepcopy(fullFrogDict)
            for frog in newFrogs:
                frog = frog.strip()
                fS = frog.split(" ")
                tempHab["N"].append(frog)
                tempFrogDict[fS[2]][fS[0] + " " + fS[1]] -= 1

            fc = fullCoverage(addFrog = newFrogs, returnKeep=True, hs = tempHab, ffd = tempFrogDict)
            tempM = [x[1] for x in fc].count("M")
            tempLen = ((len(fc) - tempM) * 2.5) + tempM
            if ((bestLen == None) or (tempLen < bestLen) or ((tempLen == bestLen) and (tempM > bestM))):
                bestLen = tempLen
                bestM = tempM
                bestSet = tables[i].loc[j, "Set Name"]
                bestSetInd = (i, j)
    return (bestSet, bestSetInd[0], "Week " + str(len(tables[bestSetInd[0]]) - bestSetInd[1]))

In [ ]:
# nextSet(space=False)
nextSet()